In [3]:
import os

StatementMeta(, 5d1aa8d0-49b9-4260-8dac-2ba686428313, 6, Finished, Available)

In [4]:
# Annotations files S01 to S10
annotations_file_names = ['Annotations-S01.parquet', 'Annotations-S02.parquet', 'Annotations-S03.parquet', 'Annotations-S04.parquet', 'Annotations-S05.parquet', 'Annotations-S06.parquet', 'Annotations-S07.parquet', 'Annotations-S08.parquet', 'Annotations-S09.parquet', 'Annotations-S10.parquet']

# Images files S01 to S10
images_file_names = ['Images-S01.parquet', 'Images-S02.parquet', 'Images-S03.parquet', 'Images-S04.parquet', 'Images-S05.parquet', 'Images-S06.parquet', 'Images-S07.parquet', 'Images-S08.parquet', 'Images-S09.parquet', 'Images-S10.parquet']

# S11 files
s11_annotations_file_name = 'Annotations-S11.parquet'
s11_images_file_name = 'Images-S11.parquet'

# Categories file
categories_file = 'Categories.parquet'

# file paths
input_dir = 'Files/metadata/parquet'
output_dir = 'Files/metadata/delta-lake'

StatementMeta(, 5d1aa8d0-49b9-4260-8dac-2ba686428313, 7, Finished, Available)

In [7]:
# read the annotations parquet files using spark and append as one dataframe
annotations_df = spark.read.parquet(os.path.join(input_dir, annotations_file_names[0]))
for file_name in annotations_file_names[1:]:
    annotations_df = annotations_df.union(spark.read.parquet(os.path.join(input_dir, file_name)))
print(f'annotations : {annotations_df.count()}')

# read the images parquet files using spark and append as one dataframe
images_df = spark.read.parquet(os.path.join(input_dir, images_file_names[0]))
for file_name in images_file_names[1:]:
    images_df = images_df.union(spark.read.parquet(os.path.join(input_dir, file_name)))
print(f'images : {images_df.count()}')

# read the categories parquet file using spark
categories_df = spark.read.parquet(os.path.join(input_dir, categories_file))
print(f'categories : {categories_df.count()}')

StatementMeta(, 5d1aa8d0-49b9-4260-8dac-2ba686428313, 10, Finished, Available)

annotations : 6755090
images : 6679039
categories : 61


In [10]:
# write the annotations dataframe to a delta lake table
annotations_df.write.format('delta').mode('overwrite').save(os.path.join(output_dir, 'annotations'))

# write the images dataframe to a delta lake table
images_df.write.format('delta').mode('overwrite').save(os.path.join(output_dir, 'images'))

# write the categories dataframe to a delta lake table
categories_df.write.format('delta').mode('overwrite').save(os.path.join(output_dir, 'categories'))

StatementMeta(, 5d1aa8d0-49b9-4260-8dac-2ba686428313, 13, Finished, Available)

In [13]:
# read and write S11 annotations and images
s11_annotations_df = spark.read.parquet(os.path.join(input_dir, s11_annotations_file_name))
s11_annotations_df.write.format('delta').mode('overwrite').save(os.path.join(output_dir, 's11annotations'))

s11_images_df = spark.read.parquet(os.path.join(input_dir, s11_images_file_name))
s11_images_df.write.format('delta').mode('overwrite').save(os.path.join(output_dir, 's11images'))

StatementMeta(, 5d1aa8d0-49b9-4260-8dac-2ba686428313, 16, Finished, Available)

In [ ]:
import json
import os
import pandas as pd

# Set the path to the directory containing the raw JSON data
raw_data_path = '/lakehouse/default/Files/raw-data'

# Get the list of JSON files in the raw data path, and select the first 10 for the training set
train_set = os.listdir(raw_data_path)[:10]

# Select the 11th file for the test set
test_set = os.listdir(raw_data_path)[10]

# Initialize empty DataFrames to store images, annotations, and categories data
images = pd.DataFrame()
annotations = pd.DataFrame()
categories = pd.DataFrame()

# Process each JSON file in the training set
for file in train_set:
    # Read the JSON file and load its data
    path = os.path.join(raw_data_path, file)
    with open(path) as f:
        data = json.load(f)

    # Extract and concatenate the 'images' and 'annotations' data into their respective DataFrames
    images = pd.concat([images, pd.DataFrame(data['images'])])
    annotations = pd.concat([annotations, pd.DataFrame(data['annotations'])])

    # The 'categories' data is the same for all files, so we only need to do it once
    if len(categories) == 0:
        categories = pd.DataFrame(data['categories'])


# Process the test set, similar to the training set
path = os.path.join(raw_data_path, test_set)
with open(path) as f:
    data = json.load(f)

# Define the paths for saving the data as Delta lake tables
train_annotations_table = 'Tables/train_annotations'
train_images_table = 'Tables/train_images'
test_annotations_table = 'Tables/test_annotations'
test_images_table = 'Tables/test_images'
categories_table = 'Tables/categories'

# Extract and convert the 'images' and 'annotations' data of the test set into DataFrames
test_images = pd.DataFrame(data['images'])
test_annotations = pd.DataFrame(data['annotations'])

# Save the DataFrames as Delta lake tables
spark.createDataFrame(images).write.format('delta').mode('overwrite').save(train_images_table)
spark.createDataFrame(annotations).write.format('delta').mode('overwrite').save(train_annotations_table)
spark.createDataFrame(test_images).write.format('delta').mode('overwrite').save(test_images_table)
spark.createDataFrame(test_annotations).write.format('delta').mode('overwrite').save(test_annotations_table)
spark.createDataFrame(categories).write.format('delta').mode('overwrite').save(categories_table)

